MLST stands for **M**ulti-**l**ocus **s**equence **t**yping. This is a framework most often used to classify bacterial isolates based on a number (ususally 7) of fragments of housekeeping genes. Briefly, each new allele of one of these gene fragments is designated as a number, and a set of 7 such numbers denotes the "sequence type" of a sample.  

In _Streptococcus pneumoniae_, MLST uses the genes aroE, gdh, gki, recP, spi, xpt, and ddl, which are all genes that code for various housekeeping enzymes.  If you downloaded and looked at the sequence-type (ST) profile data that's used here (which you can do from [PubMLST](https://pubmlst.org/bigsdb?db=pubmlst_spneumoniae_seqdef&page=plugin&name=ProfileExport)), you might see something like the following:  

ST	|   aroE	|   gdh	|   gki	|   recP	|   spi	|   xpt	|   ddl	|   clonal_complex  
----|-----------|-------|-------|-----------|-------|-------|-------|------------------
1	|   1	|   1	|   1	|   1	|   1	|   1	|   1	
2	|   1	|   1	|   4	|   1	|   18	|   13	|   18	

A useful way to estimate how related various STs might be, is to calculate the pairwise hamming distances between the allelic profiles of the STs, i.e., counting the number of alleles that are different between all the pairs of STs. The hamming distance gives us the measure of difference between a given pair of STs, and (7-hamming distance) thus tells us how similar they are. In the two STs shown above, the hamming distance is 4; the two STs differ at the alleles for gdh, spi, xpt, and ddl; and we record the similarity as 3. Doing this for all  of the STs in the data, and then only retaining those ST pairs that are identical at at least 5 loci, we can arrive at the following dataset:  

In [1]:
import pandas as pd
edges_df = pd.read_csv("mlst_dist_gt5.csv", sep = "\t", header = 0)
edges_df.head()

,source,target,weights
0,ST_1,ST_2008,6
1,ST_1,ST_7172,6
2,ST_1,ST_10475,6
3,ST_1,ST_14379,6
4,ST_2,ST_819,6


This sort of MLST similarity is a fair estimate of relatedness between the STs, and an effective way to work with this data is in the form of graphs (aka networks), where STs are the nodes, and their allelic profile similarity (as calculated above), are the weights on edges that connect the STs. `pp-netlib` is designed to drop into PopPUNK, but it can be used standalone (see [here](../../../README.md) for installation and docs) in this kind of application as well. The data this example uses is in this directory.  

Running the following code imports the `Network` class from `pp-netlib`. Then it makes a set of samples, the list of unique ST labels in the first two columns of `edges_df` (which we loaded above). Note that `edges_df` has the columns "source", "target", and "weights".  

Then, the code initialises a Network object with the `samples` set and a backend; this is the graph analysis library you would like to use. Here `pp-netlib` will use "GT", i.e. [graph-tool](https://graph-tool.skewed.de/static/doc/index.html). Finally, it constructs a network with the input data.  

In [3]:
from pp_netlib.network import Network
samples = set(edges_df["source"]).union(set(edges_df["target"]))

print(f"There are {len(samples)} unique nodes in the samples list")

## instantiate a Network object using graph-tool as the backend
mlst_network = Network(ref_list=samples, backend="GT")

## construct the network
mlst_network.construct(network_data=edges_df, weights=True)

There are 15068 unique nodes in the samples list


You can run the following call to get `pp-netlib` to print a bunch of summary statistics about the graph that's just been constructed.  

In [3]:
mlst_network.get_summary()

Network summary:
	Components				874
	Density					0.0006
	Transitivity				0.8140
	Mean betweenness			0.7434
	Weighted-mean betweenness		0.5620
	Score					0.8136
	Score (w/ betweenness)			0.2088
	Score (w/ weighted-betweenness)		0.3564




Pruning is, essentially, a way to simplify the full graph. This is useful when, for example, you want to add a new ST to this graph. In order to do this, you would want to compare the allelic profile of the new ST to that of all of the nodes in the graph. The full graph had 15,068 nodes, but the pruned version has 6473, so in this particular case, less than half the number of pairwise comparisons would be needed. Note that for very large graphs like this one, pruning can take a fair amount of time (it takes about 45 minutes for the MLST graph we are working with here).  

In [6]:
mlst_network.prune(threads=8)

Pruned network has 6473 nodes and 11256 edges.



Running the following code will calculate a minimum-spanning tree in the graph and create a couple of visualisations as .png files. These will be saved to a directory called "mst" which will be created in the working directory, along with a .graphml file of the mst network itself.  

In [4]:
mlst_network.visualise(viz_type="mst", out_prefix="mlst_mst")

Drawing MST


A variation of the `visualise` call creates outputs that you can load into cytoscape. This call also produces a file containing the list of edges, and another one containing node data (typically just the node id and the node cluster assignment). Here, you can use the `external_data` argument to merge other metadata that is associated with each node. For example, in the code below, the node data output file contains the node id and cluster, but also the allelic profile associated with that node. The node and edge data files are written through a call to `pp-netlib`'s `write_metadata` method, which can also be called directly if needed. 

In [5]:
mlst_network.visualise(viz_type="cytoscape", out_prefix="mlst_cyto", external_data="./mlst_epi_data.tsv")

For a full description of `pp-netlib` and a complete list of methods and associated arguments, see the main [README](../../../README.md).  





----

PubMLST citation: Jolley KA, Bray JE, Maiden MCJ. Open-access bacterial population genomics: BIGSdb software, the PubMLST.org website and their applications. Wellcome Open Res. 2018 Sep 24;3:124. doi: 10.12688/wellcomeopenres.14826.1. PMID: 30345391; PMCID: PMC6192448.  